In [3]:
from dataset import *

data_files = ['D:/senior/aiCourse/dataSource/comment_classification/output/train.json']
vocab_file = 'D:/senior/aiCourse/dataSource/comment_classification/output/vocab.txt'
label_file = 'D:/senior/aiCourse/dataSource/comment_classification/labels.txt'
enb_file = 'D:/senior/aiCourse/dataSource/comment_classification/embedding/embedding.txt'
batch_size = 128
reverse = False
split_word = True
max_len = 400


In [4]:
data_files=['../../../dataSource/comment_classification/output/train.json']
vocab_file = '../../../dataSource/comment_classification/output/vocab.txt'
label_file = '../../../dataSource/comment_classification/labels.txt'
enb_file = '../../../dataSource/comment_classification/output/embedding.txt'

In [5]:
dataset1 = DataSet(  data_files,  vocab_file,   label_file,   batch_size, reverse=  reverse, split_word=  split_word, max_len=  max_len)

# vocab size:  50000
# vocab size:  20
# Start to preprocessing data...
# load data from ../../../dataSource/comment_classification/output/train.json ...
# Got 105000 data items with 820 batches


In [6]:
for i,(source, lengths, targets, _) in enumerate(dataset1.get_next()):
    print(len(source[9]))
    break

402


In [7]:
def dig_lists(l):
    output = []
    for e in l:
        if isinstance(e, list):
            output += dig_lists(e)
        else:
            output.append(e)
    return(output)

In [8]:
def pad_sequences(comment_to_id,maxlen,padding,truncating):
    features = np.zeros((len(comment_to_id), maxlen), dtype=int)
    for i,comment in enumerate(comment_to_id):
        if len(comment) <= maxlen and padding == 'pre':
            features[i, -len(comment):] = np.array(comment)[:maxlen]
        if len(comment) <= maxlen and padding == 'post':
            features[i, :len(comment)] = np.array(comment)[:maxlen]
        if len(comment) > maxlen and truncating == 'post':
            features[i, :] = np.array(comment)[:maxlen]
        if len(comment) > maxlen and truncating == 'pre':
            features[i, :] = np.array(comment)[len(comment)-maxlen:]           
    return features

def split_dataset(pad_comments,labels,split_frac):
    split_index = int(len(pad_comments)*split_frac)
    data_list = list(zip(pad_comments, labels))
    random.shuffle(data_list)
    pad_comments, labels = zip(*data_list)
    x_train, x_test = pad_comments[:split_index], pad_comments[split_index:]
    y_train, y_test = labels[:split_index], labels[split_index:]
    return x_train,y_train,x_test,y_test 

In [9]:
comment_to_id = [x[0] for x in dataset1._raw_data]
pad_comments = pad_sequences(comment_to_id,maxlen=max_len,padding='post',truncating='post')

In [10]:
labels = [x[2].flatten().tolist() for x in dataset1._raw_data]

In [11]:
import random
x_train,y_train,x_test,y_test = split_dataset(pad_comments[:10000],labels,0.8)

# Model

In [27]:
import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.engine.topology import Layer
from keras import initializers as initializers, regularizers, constraints
from keras.callbacks import Callback
from keras.layers import concatenate,GlobalMaxPooling1D,GlobalAveragePooling1D,SpatialDropout1D,Embedding, Input, Dense, LSTM, GRU, Bidirectional, TimeDistributed
from keras import backend as K
from keras.models import Model
import keras.layers as layers
from sklearn.metrics import roc_auc_score
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense,Dropout,Embedding,LSTM,Bidirectional,Layer
from keras.datasets import imdb
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import *
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [13]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test) 

In [18]:
embed_size = 256
inp = Input(shape=(400, ))


In [19]:
x = Embedding(input_dim=len(dataset1.w2i), output_dim=embed_size,)(inp)
print(x)


Tensor("embedding_3/embedding_lookup/Identity:0", shape=(?, 400, 256), dtype=float32)


In [20]:
x = SpatialDropout1D(0.2)(x)

In [21]:
x1 = Bidirectional(GRU(100, return_sequences=True))(x)
print(x1)

Tensor("bidirectional_7/concat:0", shape=(?, ?, 200), dtype=float32)


In [22]:
x2 = Bidirectional(GRU(100, return_sequences=True))(x1)
print(x2)

Tensor("bidirectional_8/concat:0", shape=(?, ?, 200), dtype=float32)


In [23]:
x3 = Bidirectional(GRU(100, return_sequences=True))(x2)
print(x3)

Tensor("bidirectional_9/concat:0", shape=(?, ?, 200), dtype=float32)


In [24]:
concatenate([x1, x2,x3])

<tf.Tensor 'concatenate_1/concat:0' shape=(?, ?, 600) dtype=float32>

In [25]:
avg_pool = GlobalAveragePooling1D()(x3)
max_pool = GlobalMaxPooling1D()(x3)
concatenate([avg_pool, max_pool])

<tf.Tensor 'concatenate_2/concat:0' shape=(?, 400) dtype=float32>

In [ ]:
outp = Dense(80, activation="relu")(x3)
model = Model(inputs=x3, outputs=outp)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [33]:
def softmax(x, axis=1):
    """Softmax activation function."""
    ndim = K.ndim(x)
    if ndim == 2:
        return K.softmax(x)
    elif ndim > 2:
        e = K.exp(x - K.max(x, axis=axis, keepdims=True))
        s = K.sum(e, axis=axis, keepdims=True)
        return e / s
    else:
        raise ValueError('Cannot apply softmax to a tensor that is 1D')
        
        
densor1 = Dense(32, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)
def one_step_attention(a):
    e = densor1(a)
    energies = densor2(e)
    alphas = activator(energies)
    context = dotor([alphas,a])
    return context

def get_model():

    inp = Input(shape=(400,))
    x = Embedding(50000, 256)(inp)
    x = Bidirectional(CuDNNGRU(64, return_sequences= True))(x)
    x = Dropout(0.25)(x)
    context = one_step_attention(x)
    context = Flatten()(context)
    merged = Dropout(0.25)(context)
    merged = BatchNormalization()(merged)
    preds = Dense(80, activation='relu')(merged)
    model = Model(inputs = [inp], outputs= preds)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    return model

In [16]:
def get_model():
    embed_size = 256
    inp = Input(shape=(400, ))
    x = Embedding(input_dim=len(dataset1.w2i), output_dim=embed_size,)(inp)
    print(x)
    x = SpatialDropout1D(0.2)(x)
    x1 = Bidirectional(GRU(100, return_sequences=True))(x)
    x2 = Bidirectional(GRU(100, return_sequences=True))(x1)
    x3 = Bidirectional(GRU(100, return_sequences=True))(x2)
        
    outp = Dense(80, activation="relu")(x3)
    model = Model(inputs=x3, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model
    
    avg_pool = GlobalAveragePooling1D()(x3)
    max_pool = GlobalMaxPooling1D()(x3)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(80, activation="relu")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [34]:
model = get_model()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 400)          0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 400, 256)     12800000    input_6[0][0]                    
__________________________________________________________________________________________________
bidirectional_11 (Bidirectional (None, 400, 128)     123648      embedding_5[0][0]                
__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 400, 128)     0           bidirectional_11[0][0]           
__________________________________________________________________________________________________
dense_5 (D

In [35]:
print(model.summary())
history = model.fit(x_train, y_train,epochs=2,verbose=1, batch_size=batch_size)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 400)          0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 400, 256)     12800000    input_6[0][0]                    
__________________________________________________________________________________________________
bidirectional_11 (Bidirectional (None, 400, 128)     123648      embedding_5[0][0]                
__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 400, 128)     0           bidirectional_11[0][0]           
__________________________________________________________________________________________________
dense_5 (D

InvalidArgumentError: No OpKernel was registered to support Op 'CudnnRNN' with these attrs.  Registered devices: [CPU], Registered kernels:
  <no registered kernels>

	 [[node bidirectional_10/CudnnRNN (defined at e:\anaconda\envs\aicourse\lib\site-packages\tensorflow\contrib\cudnn_rnn\python\ops\cudnn_rnn_ops.py:922)  = CudnnRNN[T=DT_FLOAT, direction="unidirectional", dropout=0, input_mode="linear_input", is_training=true, rnn_mode="gru", seed=87654321, seed2=0](bidirectional_10/transpose, bidirectional_10/ExpandDims_1, bidirectional_10/Const, bidirectional_10/concat)]]

Caused by op 'bidirectional_10/CudnnRNN', defined at:
  File "e:\anaconda\envs\aicourse\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "e:\anaconda\envs\aicourse\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "e:\anaconda\envs\aicourse\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "e:\anaconda\envs\aicourse

In [31]:
model.predict(x_train[:1])

array([[0.27148858, 0.        , 0.03656801, 0.49509174, 0.16101427,
        0.        , 0.        , 0.4964216 , 0.2141101 , 0.01124754,
        0.        , 0.        , 0.04661782, 0.04072843, 0.19887982,
        0.43112528, 0.37832567, 0.10584863, 0.        , 0.        ,
        0.        , 0.        , 0.08206163, 0.39976266, 0.        ,
        0.        , 0.        , 0.48179057, 0.15471175, 0.1760441 ,
        0.23459688, 0.47964907, 0.24099392, 0.        , 0.04933136,
        0.4643405 , 0.18672746, 0.22675116, 0.01986222, 0.5704697 ,
        0.29397368, 0.3688913 , 0.        , 0.7803701 , 0.3242618 ,
        0.0779756 , 0.03781231, 0.4317289 , 0.23610915, 0.14585766,
        0.06815854, 0.47883025, 0.2631844 , 0.07373718, 0.04461501,
        0.4500302 , 0.40371394, 0.        , 0.        , 0.4848981 ,
        0.4600464 , 0.33543804, 0.04280333, 0.02706825, 0.23082788,
        0.06336311, 0.        , 0.4076091 , 0.2201471 , 0.01970156,
        0.03127155, 0.6290768 , 0.        , 0.31

def get_model():
    embed_size = 256
    model = Sequential()   
    
    model.add( Embedding(50000, embed_size,input_length=400))
    
    
    model.add(Bidirectional(GRU(100, return_sequences=False)))
    #model.add(SpatialDropout1D(0.2))
              
    #model.add(Bidirectional(GRU(100, return_sequences=True)))
    
    #model.add(GlobalAveragePooling1D())
    model.add(Dense(80, activation="relu"))
    
    
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [ ]:
import matplotlib.pyplot as plt
print(history.history.keys())
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

preds = model.predict(x_train)
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])
print(accuracy(preds,y_train))

In [ ]:
def convert_2_labels(answer):
    labels_input = []
    for item in answer:
        answer_temp = []
        for i in range(0,80,4):
            answer_temp.append(np.argmax(item[i:i+4]))
        labels_input.append(answer_temp)
    return labels_input



In [ ]:
convert_2_labels(preds)

In [ ]:
coun=0
all1=0
for line in convert_2_labels(y_train):
    for item in line:
        if item == 3:
            coun+=1
        all1+=1